In [ ]:
!pip install transformers

In [ ]:
!pip install --upgrade transformers

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
!mkdir my_data
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" -o my_data/wiki_20190620_small.txt

path = "/content/my_data/wiki_20190620_small.txt"

mkdir: cannot create directory ‘my_data’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1323k  100 1323k    0     0  1500k      0 --:--:-- --:--:-- --:--:--  107M


`SentencePieceBPETokenizer`는 띄어쓰기에 자동으로 '_'를 추가해줌

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install protobuf

In [ ]:
from tokenizers import SentencePieceBPETokenizer
from tokenizers.normalizers import BertNormalizer
from transformers import AlbertTokenizer

tokenizer=AlbertTokenizer.from_pretrained('albert-base-v2',use_fast=True)
'''
tokenizer._tokenizer.normalizer=BertNormalizer(clean_text=True,
                                               handle_chinese_chars=False,
                                               lowercase=False)

tokenizer.train(
    path,
    vocab_size=10000,
    special_tokens=[
        '<s>', # 문장의 시작
        '<pad>', # 패딩 토큰
        '</s>', # 문장의 끝
        '<unk>' # 사전에 없는 토큰
    ],
)
'''

"\ntokenizer._tokenizer.normalizer=BertNormalizer(clean_text=True,\n                                               handle_chinese_chars=False,\n                                               lowercase=False)\n\ntokenizer.train(\n    path,\n    vocab_size=10000,\n    special_tokens=[\n        '<s>', # 문장의 시작\n        '<pad>', # 패딩 토큰\n        '</s>', # 문장의 끝\n        '<unk>' # 사전에 없는 토큰\n    ],\n)\n"

In [ ]:
print(tokenizer.encode('이순신은 조선 중기의 무신이다'))
print(tokenizer.encode('이순신은 조선 중기의 무신이다').ids)
print(tokenizer.encode('이순신은 조선 중기의 무신이다').tokens)
print(tokenizer.decode(tokenizer.encode('<s>이순신은 조선 중기의 무신이다.</s>').ids,skip_special_tokens=True))
# sentencepiece를 사용하면, 나중에 decoding 과정에서 '_'로 replace해주면 띄어쓰기 복원이 가능해짐

[2, 13, 1, 13, 1, 13, 1, 13, 1, 3]


AttributeError: 'list' object has no attribute 'ids'

- 띄어쓰기(어절의 시작)에는 "_"가 생성이 되고, 토큰별로 잘 변환된 것을 확인할 수 있습니다.
- 그러나 토크나이저에 추가했던 스페셜 토큰이 제대로 작동하지 않음

In [ ]:
print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").tokens)

['<s>', '▁이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.', '</s>']


In [ ]:
tokenizer.add_special_tokens(['<s>','</s>','<unk>','<pad>'])
tokenizer.pad_token_id = tokenizer.token_to_id("<pad>")
tokenizer.unk_token_id = tokenizer.token_to_id("<unk>")
tokenizer.bos_token_id = tokenizer.token_to_id("<s>")
tokenizer.eos_token_id = tokenizer.token_to_id("</s>")

In [ ]:
print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").tokens)


['<s>', '▁이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.', '</s>']


In [ ]:
tokenizer.save("tokenizer.json")

In [ ]:
from transformers import PreTrainedTokenizerFast
PreTrainedTokenizerFast.from_pretrained("tokenizer.json")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1985: FutureWarning: Calling PreTrainedTokenizerFast.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


PreTrainedTokenizerFast(name_or_path='tokenizer.json', vocab_size=10000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
from transformers import PreTrainedTokenizerFast

#your code
tokenizer.save_model(".")
tokenizer = PreTrainedTokenizerFast(vocab_filename="vocab.json", merges_filename="merges.txt")

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

GPT 학습 준비

In [ ]:
from transformers import GPT2Config, GPT2LMHeadModel

config=GPT2Config(
    vocab_size=tokenizer.get_vocab_size(),
    bos_token_id=tokenizer.token_to_id('<s>'), # 문장의 시작
    eos_token_id=tokenizer.token_to_id('</s>') # 문장의 끝
)

model=GPT2LMHeadModel(config)

In [ ]:
import json
import os
import pickle
import random
import time
import warnings
from typing import Dict, List, Optional

import torch
from torch.utils.data.dataset import Dataset

from filelock import FileLock

from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.utils import logging

In [ ]:
logger = logging.get_logger(__name__)

class TextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        block_size = block_size - tokenizer.num_special_tokens_to_add(is_pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")
                # 여기서부터 본격적으로 데이터셋을 만들기 시작합니다.
                self.examples = []
                text = ""
                with open(file_path, encoding="utf-8") as f:
                    lines = f.readlines()
                    for line in lines:
                        line = line.strip()
                        line = "<s>"+line+"</s>" # 학습 데이터 앞 뒤에 문장 구분 기호를 추가해줍니다.
                        text += line    # 'text' 객체에 모든 학습 데이터를 다 합쳐버립니다 :-)
                tokenized_text = tokenizer.encode(text).ids

                # 모델의 최대 sequence length만큼 데이터를 잘라서 저장합니다.
                for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                    self.examples.append(
                        tokenized_text[i : i + block_size]
                    )
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should look for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=path,
    block_size=128,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)

- GPT는 생성모델이기 때문에 [MASK]가 필요없음
- [MASK] 토큰은 주어진 문장에서 특정 토큰을 가리고 이를 복원하는 작업을 수행하는데, GPT는 문맥을 보고 다음 단어를 예측하는 방식이기 때문에 MASK 토큰이 필요하지 않음

In [ ]:
!pip install -U accelerate
!pip install -U transformers

In [ ]:
import transformers
import accelerate

transformers.__version__, accelerate.__version__

('4.39.3', '0.29.1')

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=64, # 512:32  # 128:64
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


AttributeError: 'SentencePieceBPETokenizer' object has no attribute 'pad_token_id'